# Data Cleaning
delete files in manually labeled data (**release-level-data** from replication kit https://zenodo.org/record/5675024#.Ya-B8fHML0o) that are not related to bug fixes

In [55]:
import os

for filename in os.listdir('./release-level-data'):
    if filename.endswith("bug_fixes.json"):
        continue
    os.remove('./release-level-data/' + filename)
print("data cleaning done")

data cleaning done


# Extract traditional smells
returns a dictionary of list, with key represents a traditional smell and value represents list of classes that contain the smell

In [46]:
import os
import subprocess
import configparser
import pandas as pd
import re
import shutil
import time
import sys

def extract_traditional_smells(project_name):
    print("extracting traditional smells from repository: " + project_name)
    subprocess.call(['java', '-jar', './DECOR_JAVA.jar', project_name, project_name, ""])

    # check if the result of detection is generated
    if os.path.isdir('../TEMP'):

        # dictionary of list, key represents a traditional smell and value represents list of classes that contain the smell
        smell_classes_dict = {}
        # iterate through each generated file
        result_files_path = "../TEMP/" + project_name
        for filename in os.listdir(result_files_path):

            # read the content of the file
            file_path = result_files_path + "/" + filename
            with open(file_path) as file:
                file_content = '[dummy_section]\n' + file.read()
            config = configparser.ConfigParser()
            config.read_string(file_content)

            # find all smelly test classes that contain traditional smell
            smelly_classes = []
            for key, value in config.items('dummy_section'):
                if re.match("^[0-9]+\.[0-9]+\.[a-zA-Z]+-\d$", key):
                    class_name = value.split(".")[-1]
                    smelly_classes.append(class_name)
            # map smell and classes containing the smell
            if smelly_classes:
                smell_name = filename.split(" ")[-1].split(".")[0]
                smell_classes_dict[smell_name] = smelly_classes

        # delete result of detection
        try:
            shutil.rmtree("../TEMP/")
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))
        finally:
            print("Traditional smells extracted for project: ", project_name)
            return smell_classes_dict

    else:    
        print("ERROR: extracting traditionial smells failed")
        return None

# Extract traditional smells test


In [49]:
smell_classes_dict = extract_traditional_smells('ant-ivy')
print(smell_classes_dict)

extracting traditional smells from repository: ant-ivy
Traditional smells extracted for project:  ant-ivy
{'LazyClass': ['IvyWebdavFileSystem', 'IvyWebdavFileSystem'], 'LongMethod': ['VfsResourceTest', 'XmlModuleUpdaterTest', 'IvyCachePathTest', 'TestPerformance', 'IvyCacheTask', 'ModuleRevisionId', 'IvyVar', 'ResolveTest', 'CredentialsUtil', 'IBiblioResolverTest', 'SortTest', 'IvyArtifactReport', 'IvyInstall', 'VfsRepository', 'IvyTask', 'JarJarDependencyAnalyser', 'IvyWebdavClientFactory', 'VersionRangeMatcher', 'FileUtil', 'MRIDTransformationRuleTest', 'AbstractSshBasedRepository', 'SshRepository', 'Main', 'AntBuildTrigger', 'IvyReport', 'IvyBuildNumber', 'IvyCacheFileset', 'DualResolverTest', 'IvyPublishTest', 'ConfiguratorTest', 'IvyWebdavConnectionManager', 'ApacheURLLister', 'Message', 'JarModuleFinder', 'ChainResolverTest', 'IvyConfigure', 'DefaultModuleDescriptor', 'RepositoryResolver', 'IvyListModules', 'IvyInfo', 'SFTPRepository', 'VfsRepositoryTest', 'BasicURLHandler', 'IBi

# Extract traditional smells 2
returns a set of classes that contain a smell

In [34]:
import os
import subprocess
import configparser
import pandas as pd
import re
import shutil
import time
import sys

def extract_traditional_smells2(project_name):
    print("extracting traditional smells from repository: " + project_name)
    subprocess.call(['java', '-jar', './DECOR_JAVA.jar', project_name, project_name, ""])

    # check if the result of detection is generated
    if os.path.isdir('../TEMP'):

        # set of classes that contain the smell
        smell_classes_set = set()

        # iterate through each generated file
        result_files_path = "../TEMP/" + project_name
        for filename in os.listdir(result_files_path):

            # read the content of the file
            file_path = result_files_path + "/" + filename
            with open(file_path) as file:
                file_content = '[dummy_section]\n' + file.read()
            config = configparser.ConfigParser()
            config.read_string(file_content)

            # find all smelly test classes that contain traditional smell
            for key, value in config.items('dummy_section'):
                if re.match("^[0-9]+\.[0-9]+\.[a-zA-Z]+-\d$", key):
                    class_name = value.split(".")[-1]
                    smell_classes_set.add(class_name)

        # delete result of detection
        try:
            shutil.rmtree("../TEMP/")
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))
        finally:
            print("Traditional smells extracted for project: ", project_name)
            return smell_classes_set

    else:    
        print("ERROR: extracting traditionial smells failed")
        return None

# RQ1 - Fisher exact test

In [30]:
from git import Repo
import json
from scipy.stats import fisher_exact

# clone repo
# repo = Repo.clone_from('https://github.com/apache/ant-ivy', 'ant-ivy')
repo = Repo('ant-ivy')

# checkout to specific release
# print(repo.tags)
repo.git.checkout('1.4.1')

smelly_classes = extract_traditional_smells2('ant-ivy')
print(smelly_classes)

# number of classes with at least one fixing and with at least one anti-pattern 
w_fix_w_anti = 0
# number of classes with at least one fixing and without any anti-patterns
w_fix_wo_anti = 0
# number of classes without any fixing and with at least one anti-pattern 
wo_fix_w_anti = 0
# number of classes without any fixing and without any anti-patterns
wo_fix_wo_anti = 0

f = open('./release-level-data/ant-ivy-1.4.1_bug_fixes.json')
data = json.load(f)
for line in data:
    filename = line['file'].split("/")[-1].split(".")[0]
    if line['bug_fixes']:
        if filename in smelly_classes:
            w_fix_w_anti +=1
        else:
            w_fix_wo_anti += 1
    else:
        if filename in smelly_classes:
            wo_fix_w_anti +=1
        else:
            wo_fix_wo_anti += 1

f.close()

# computer fisher exact test
data = [[w_fix_w_anti, wo_fix_w_anti],
         [w_fix_wo_anti, wo_fix_wo_anti]]

oddsratio, pvalue = fisher_exact(data)
print('\n')
print('Fisher exact test odds ratio: {:.4f}'.format(oddsratio))
print('Fisher exact test p-value: {:.4f}'.format(pvalue))

extracting traditional smells from repository: ant-ivy
Traditional smells extracted for project:  ant-ivy
{'CredentialsUtil', 'FileUtil', 'XmlReportOutputter', 'DefaultDependencyDescriptor', 'AbstractModuleDescriptorParser', 'SortTest', 'JarModuleFinder', 'IvyRepResolverTest', 'IvyCachePathTest', 'VersionRangeMatcher', 'IvyRetrieveTest', 'IvyInstall', 'CacheResolver', 'ModuleDescriptorSorter', 'ResolveReport', 'IvyPublish', 'IvyBuildNumber', 'URLResolverTest', 'LatestRevisionStrategy', 'URLRepository', 'XmlModuleDescriptorParser', 'ResolveData', 'MockResolver', 'IvyListModules', 'IvyExtractFromSources', 'StatusManager', 'DefaultDependencyArtifactDescriptor', 'BasicResolver', 'BasicURLHandler', 'IvyWebdavConnectionManager', 'ChainResolverTest', 'XmlModuleUpdaterTest', 'IvyTask', 'Scp', 'XmlModuleDescriptorUpdater', 'IvyDeliver', 'IvyVar', 'DefaultModuleDescriptor', 'IvyPublishTest', 'DefaultArtifact', 'IvyConfigure', 'AbstractSshBasedRepository', 'IvyCheck', 'IvyCacheFileset', 'SshCache